In [1]:
import sklearn
import math
import pandas
import numpy
from sklearn.linear_model import LinearRegression

In [2]:
df = pandas.DataFrame({"x": numpy.random.rand(100)})
df["y"] = -df["x"] * df["x"] + 4
df["z"] = df["y"] * 2 + df["x"]

In [3]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space = [
    Integer(0, 1, name="intercept"),
    Real(0, 1, "uniform", name="dummy"),
    Real(1, 5, "uniform", name="scale"),    
]

In [4]:
from skopt import gp_minimize
from skopt import forest_minimize

class MLModel(object):
    def __init__(self, df, params={}):
        self.params = params
        self.df = df
    
    def set_params(self, **params):
        print(params)
        self.params = params

    def set_dataftame(self, df):
        self.df = df
        
    def opt_fit(self, space):
        #gp = gp_minimize(self.objective, space, n_calls=50, random_state=0)
        
        @use_named_args(space)
        def my_objective(**params):
            self.set_params(**params)
            obj = self._train()
            return -obj
            
        gp = forest_minimize(func=my_objective, dimensions=space)
        return gp
       
    def objective(self, **params):
        self.set_params(**params)
        obj = self._train()
        return obj
        
    def _train(self):
        df = self.df.copy()
        lm = LinearRegression(fit_intercept=self.params["intercept"])
        df["z"] = df["z"] + numpy.random.rand(len(df)) * self.params["scale"] + self.params["dummy"]
        lm.fit(df[["x", "y"]], df["z"])
        obj = lm.score(df[["x", "y"]], df["z"])
        return obj

In [5]:
from skopt import forest_minimize

In [6]:
ml = MLModel(df)

In [7]:
ml.set_params(intercept=0,dummy=0,scale=1)
ml._train()

{'intercept': 0, 'dummy': 0, 'scale': 1}


0.5098233518143205

In [8]:
ml.set_params(intercept=1,dummy=0,scale=1)
ml._train()

{'intercept': 1, 'dummy': 0, 'scale': 1}


0.6314855135036538

In [9]:
g = ml.opt_fit(space)

{'intercept': 1, 'dummy': 0.6734732374690231, 'scale': 4.88305474116823}
{'intercept': 1, 'dummy': 0.5780404140064558, 'scale': 4.0841913517478945}
{'intercept': 0, 'dummy': 0.5360229582726452, 'scale': 1.9293580229004832}
{'intercept': 0, 'dummy': 0.41350281197698746, 'scale': 2.753398670598337}
{'intercept': 1, 'dummy': 0.05873487834956738, 'scale': 2.4018445576502447}
{'intercept': 0, 'dummy': 0.1412962994991447, 'scale': 2.420755874370882}
{'intercept': 0, 'dummy': 0.5303695501762417, 'scale': 1.850273928658677}
{'intercept': 1, 'dummy': 0.24488471672035234, 'scale': 2.3405413094595886}
{'intercept': 0, 'dummy': 0.34885152033299877, 'scale': 4.197724026590214}
{'intercept': 1, 'dummy': 0.9027053810865613, 'scale': 3.5397404164747868}
{'intercept': 0, 'dummy': 0.5466861067747851, 'scale': 1.843894198832207}
{'intercept': 0, 'dummy': 0.5457575734734014, 'scale': 1.971759563087249}
{'intercept': 0, 'dummy': 0.5736733604050993, 'scale': 1.2671081717341885}
{'intercept': 0, 'dummy': 0.6

In [10]:
g.fun

-0.6204017546688616

In [11]:
g.x

[1, 0.05966279534429898, 1.0997025864547059]

In [12]:
g

          fun: -0.6204017546688616
    func_vals: array([-0.14089768, -0.09751788, -0.32343801, -0.0950523 , -0.13911493,
       -0.08778427, -0.34345629, -0.18187675, -0.03184358, -0.10011997,
       -0.348675  , -0.20710654, -0.52791703, -0.36761409, -0.56340279,
       -0.39794438, -0.4972076 , -0.50095337, -0.53798282, -0.5156058 ,
       -0.42505669, -0.52333267, -0.59911645, -0.54708982, -0.47377502,
       -0.50965261, -0.60807198, -0.53257316, -0.39631441, -0.55353968,
       -0.48331143, -0.41545321, -0.61165976, -0.53949768, -0.45079196,
       -0.48470926, -0.50867634, -0.45766874, -0.52927111, -0.5763731 ,
       -0.54966511, -0.58589213, -0.49420809, -0.54400478, -0.50754247,
       -0.36462827, -0.53007614, -0.41918812, -0.42430565, -0.51594839,
       -0.33979374, -0.46521702, -0.50791027, -0.52003415, -0.46659931,
       -0.50355848, -0.49077374, -0.52974505, -0.3910879 , -0.48640568,
       -0.42312519, -0.47283742, -0.57411031, -0.53419809, -0.55136741,
       -0.5320